In [ ]:
import os
import pandas as pd
import yaml
import time
from github import Github

# Repository URL
repo_url = "ethereum-optimism/superchain-registry"

# GitHub access token
access_token = os.environ['GH_PUBLIC_TOKEN']


In [ ]:
# Function to extract data from YAML file
# Function to extract data from YAML file
def extract_data_from_yaml(content):
    data = yaml.safe_load(content)
    return data

# Function to extract YAML content and create additional columns
def extract_yaml_content_and_columns(file_content):
    yaml_data = extract_data_from_yaml(file_content)
    flattened_data = flatten_dict(yaml_data)
    return flattened_data

# Function to create DataFrame
def create_dataframe(repo):
    data = []
    contents = repo.get_contents("superchain/configs", ref="main")
    for content in contents:
        if content.type == "dir":
            network_version = content.name
            files = repo.get_contents(f"superchain/configs/{network_version}", ref="main")
            for file in files:
                if file.name.endswith(".yml") or file.name.endswith(".yaml"):
                    chain_name = os.path.splitext(os.path.basename(file.name))[0]
                    if "superchain" not in chain_name and "semver" not in chain_name:
                        yaml_content = file.decoded_content.decode("utf-8")
                        row = {"network_version": network_version, "chain_name": chain_name}
                        row.update(extract_yaml_content_and_columns(yaml_content))
                        data.append(row)
    return pd.DataFrame(data)

# Function to flatten a nested dictionary
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [ ]:
# Authenticate with GitHub
g = Github(access_token)
# Get the repository
repo = g.get_repo(repo_url)
print(repo)

In [ ]:
df = create_dataframe(repo)
# Generate Cols
df["chain_config_name"] = df["network_version"] + "_" + df["chain_name"]
df["display_name"] = df["name"].str.replace("_", " ").str.replace("-", " ")
# Print DataFrame
### TODO
### Pull Batcher / Bridge / All other addresses discoverable from the system config proxy
### This will require loading in the Ecotone system config ABI & Calling the functions.

In [ ]:
df

In [ ]:
# Save Dataframe
df.to_csv('outputs/superchain-registry.csv')